In [3]:
import pandas as pd

df = pd.read_csv('/content/IMDB Dataset.csv')
display(df.head())
display(df.info())

ParserError: Error tokenizing data. C error: EOF inside string starting at row 4762

In [ ]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

df['cleaned_review'] = df['review'].apply(clean_text)
display(df.head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,review,sentiment,cleaned_review
0,One of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching oz episode yo...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production br br filming tech...
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,basically theres family little boy jake thinks...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df['cleaned_review'])
y = df['sentiment']

print("Shape of TF-IDF matrix (X):", X.shape)
print("Shape of target variable (y):", y.shape)

Shape of TF-IDF matrix (X): (50000, 175731)
Shape of target variable (y): (50000,)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = MultinomialNB()

model.fit(X_train, y_train)

print("Model training complete.")

Model training complete.


**Reasoning**:
Import necessary metrics, make predictions on the test set, calculate evaluation metrics, and print the results.



In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='positive')
recall = recall_score(y_test, y_pred, pos_label='positive')
f1 = f1_score(y_test, y_pred, pos_label='positive')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 0.8690
Precision: 0.8832
Recall: 0.8527
F1-score: 0.8677


In [ ]:
from ipywidgets import Textarea, Button, VBox, Output, Label
from IPython.display import display

# Create widgets
review_input = Textarea(placeholder='Enter your movie review here...', layout={'width': '500px', 'height': '100px'})
predict_button = Button(description='Predict Sentiment')
output_area = Output()
label = Label("Enter a movie review to predict its sentiment:")

# Function to handle prediction
def on_predict_button_clicked(b):
    with output_area:
        output_area.clear_output()
        new_review = review_input.value
        if new_review:
            cleaned_review = clean_text(new_review)
            vectorized_review = tfidf_vectorizer.transform([cleaned_review])
            prediction = model.predict(vectorized_review)[0]
            print(f"Review: {new_review}\nPredicted Sentiment: {prediction}\n")
        else:
            print("Please enter a review.")

# Link button click to function
predict_button.on_click(on_predict_button_clicked)

# Display widgets
display(VBox([label, review_input, predict_button, output_area]))